In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from transformers import pipeline

import warnings
warnings.filterwarnings("ignore")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.set_option('display.max_columns', None) 

In [4]:
from google.colab import files
uploaded = files.upload()

Saving HandLabel_fixed.csv to HandLabel_fixed.csv


In [6]:
df = pd.read_csv('HandLabel_fixed.csv', index_col = [0])

In [7]:
df.head()

,segment,review_id,product_name,producturl,retailer,brand,retailer_product_id,base_id,variants_flag,review_creation_date,year,month,week,review_title,review_text,review_type,price,rating,images,videos,helpful_votes,promotion_flag,review_lower,date,word_count,sentiment,cost,ease of use,effectiveness,efficiency,Unnamed: 31
0,Garden,52479,AMDRO Fire Ant Bait 1lb,https://www.lowes.com/pd/AMDRO-1-lb-Fire-Ant-B...,Lowes,Amdro,4573846,3006355,Yes,2021-08-31T00:00:00,2021,8,35,Try a little before you use the entire thing,It didnt work. The ants didnt even bother eati...,organic reviews,10.07,1,0,0,0,No,it didnt work. the ants didnt even bother eati...,2021-08-31,10,NaN,NaN,NaN,NaN,NaN,NaN
1,Garden,24208,AMDRO Mouse Trap,https://www.homedepot.com/p/AMDRO-Mouse-Trap-1...,Homedepot,Amdro,306335043,306335043,No,2021-09-08T00:00:00,2021,9,36,Only one reason for not giving five stars,This mouse trap works phenomenally! I had trie...,syndicated reviews,6.98,4,0,0,0,No,this mouse trap works phenomenally! i had trie...,2021-09-08,92,positive,NaN,NaN,1.0,1.0,NaN
2,Garden,141806,"AMDRO Quick Kill Carpenter Bee, Ant & Termite ...",http://www.homedepot.com/p/305448729,Homedepot,Amdro,305448910,305448729,Yes,2022-06-10T00:00:00,2022,6,23,Excellent product! I normally have to battle k...,Excellent product! I normally have to battle k...,organic reviews,10.46,5,0,0,0,No,excellent product! i normally have to battle k...,2022-06-10,62,positive,NaN,NaN,1.0,NaN,NaN
3,Garden,130119,AMDRO Quick Kill Mosquito Bombs,https://www.amazon.com/dp/B07G1YXZ9Y,Amazon,Amdro,B07G1YXZ9Y,B07G1YXZ9Y,No,2022-05-27T00:00:00,2022,5,21,Not killing larvae,Ive purchased these before and they have worke...,organic reviews,9.93,1,0,0,0,No,ive purchased these before and they have worke...,2022-05-27,16,negative,NaN,NaN,1.0,NaN,NaN
4,Garden,68528,AMDRO Ant Block Home Perimeter Ant Bait,https://www.homedepot.com/p/AMDRO-12-oz-Ant-Bl...,Homedepot,Amdro,305448482,100598633,Yes,2021-07-09T00:00:00,2021,7,27,Works great on all kinds of ants. Even takes o...,Works great on all kinds of ants. Even takes o...,organic reviews,8.62,5,0,0,1,No,works great on all kinds of ants. even takes o...,2021-07-09,12,positive,NaN,NaN,1.0,NaN,NaN


In [8]:
#replace NaN in label columns with 0
columns = ['cost','ease of use', 'effectiveness', 'efficiency']

for column in columns:
    df[column] = df[column].replace(np.nan, int(0))

In [9]:
classifier  = pipeline("zero-shot-classification",  model = "facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
# insert the labels you identified from above section
type = ['cost', 'efficiency', 'effectiveness', 'ease of use']

In [ ]:
#setting empty values for the columns
index = -1
for label in type:
    df[label + '_m'] = -1

for j in range(99):
    #counter for progress/debugging
    index+=1
    if(index%20 == 0): 
        print(index)
        
    #running the classifier on the column    
    res = classifier(
        df.iloc[j]['review_lower'],
        candidate_labels = type,
        multi_label = True
    )
    #setting the column values according to the output from the classifier ("_m" = multiclass)
    for i in range(len(res['labels'])):
        df[res['labels'][i]+ '_m'].iloc[j] = res['scores'][i]

0
20
40
60


In [13]:
from sklearn.metrics import classification_report

In [14]:
def imputation(list):
    i = []
    for prediction in list:
        if prediction < 0.7:
            i.append(0)
        else:
            i.append(1)
    
    return(i)

In [16]:
df['cost_m'] = imputation(df['cost_m'])
df['efficient_m'] = imputation(df['efficiency_m'])
df['effective_m'] = imputation(df['effectiveness_m'])
df['ease of use_m'] = imputation(df['ease of use_m'])

In [17]:
print(classification_report(df['cost'], df['cost_m']))

              precision    recall  f1-score   support

         0.0       1.00      0.95      0.97       489
         1.0       0.29      1.00      0.44        10

    accuracy                           0.95       499
   macro avg       0.64      0.97      0.71       499
weighted avg       0.99      0.95      0.96       499



In [18]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(df['cost'], df['cost_m'])
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.9744


In [21]:
print(classification_report(df['efficiency'], df['efficient_m']))

              precision    recall  f1-score   support

         0.0       0.97      0.95      0.96       461
         1.0       0.48      0.61      0.53        38

    accuracy                           0.92       499
   macro avg       0.72      0.78      0.75       499
weighted avg       0.93      0.92      0.92       499



In [23]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(df['efficiency'], df['efficient_m'])
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.7755


In [24]:
print(classification_report(df['effectiveness'], df['effective_m']))

              precision    recall  f1-score   support

         0.0       0.97      0.96      0.96       430
         1.0       0.77      0.78      0.78        69

    accuracy                           0.94       499
   macro avg       0.87      0.87      0.87       499
weighted avg       0.94      0.94      0.94       499



In [25]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(df['effectiveness'], df['effective_m'])
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.8727


In [26]:
print(classification_report(df['ease of use'], df['ease of use_m']))

              precision    recall  f1-score   support

         0.0       1.00      0.94      0.97       487
         1.0       0.27      0.92      0.42        12

    accuracy                           0.94       499
   macro avg       0.63      0.93      0.69       499
weighted avg       0.98      0.94      0.95       499



In [27]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(df['ease of use'], df['ease of use_m'])
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.9275


In [28]:
#df.to_csv('testing_evaluation.csv')

In [29]:
#df2 = pd.read_csv('testing_evaluation.csv', index_col = [0])

In [30]:
#evaluation(df2['cost'], df2['naive_cost'])

NameError: ignored

In [ ]:
# from sklearn.metrics import roc_auc_score
# ROC_AUC = roc_auc_score(df2['cost'], df2['naive_cost'])
# print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.5000


In [ ]:
#evaluation(df2['efficient'], df2['naive_efficient'])

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       375
           1       0.00      0.00      0.00        25

    accuracy                           0.94       400
   macro avg       0.47      0.50      0.48       400
weighted avg       0.88      0.94      0.91       400



In [ ]:
# from sklearn.metrics import roc_auc_score
# ROC_AUC = roc_auc_score(df2['efficient'], df2['naive_efficient'])
# print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.5000


In [ ]:
#evaluation(df2['effective'], df2['naive_effective'])

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       316
           1       0.21      1.00      0.35        84

    accuracy                           0.21       400
   macro avg       0.10      0.50      0.17       400
weighted avg       0.04      0.21      0.07       400



In [ ]:
# from sklearn.metrics import roc_auc_score
# ROC_AUC = roc_auc_score(df2['effective'], df2['naive_effective'])
# print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.5000


In [ ]:
#evaluation(df2['ease of use'], df2['naive_easeofuse'])

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       376
           1       0.00      0.00      0.00        24

    accuracy                           0.94       400
   macro avg       0.47      0.50      0.48       400
weighted avg       0.88      0.94      0.91       400



In [ ]:
# from sklearn.metrics import roc_auc_score
# ROC_AUC = roc_auc_score(df2['ease of use'], df2['naive_easeofuse'])
# print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.5000
